In [6]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [7]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [8]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""


In [9]:
usecase_name = "Learning days history"
project_name = "MatchaEnglishWebsite"
main_flow_prompt = """
basic flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system redirects learner to History page.
Step 3: Learner selects "Day History" on History page.
Step 4: The system redirects learner to a page show how many days learner has learned continuously in number. If learner break their chain, the number reset to one.

"""
prompt_all = """
basic flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system redirects learner to History page.
Step 3: Learner selects "Day History" on History page.
Step 4: The system redirects learner to a page show how many days learner has learned continuously in number. If learner break their chain, the number reset to one.
"""
alt_prompt=[]
exc_prompt = []

In [10]:
for alt in alt_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+alt}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(alt)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")
for exc in exc_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+exc}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(exc)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")

  print()
  print()
promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
sub_gpt_response = ask(promptSubScenario, client, model)
print(prompt_all)
print("\all flow gen sub flow:"+sub_gpt_response +"\n")


basic flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system redirects learner to History page.
Step 3: Learner selects "Day History" on History page.
Step 4: The system redirects learner to a page show how many days learner has learned continuously in number. If learner break their chain, the number reset to one.

ll flow gen sub flow:Based on the provided information, there is only one main flow described without any alternative or exception flow explicitly mentioned. Therefore, I will assume an alternative or exception flow based on a common scenario that could occur in such a system. 

Assuming an alternative or exception flow could be:

Alternative flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system fails to redirect learner to History page due to a network error or system issue.

Given the rules for predicting scenarios, here are the scenarios that can lead a user from the main flow to change to the alternative or exception flow:

1. **Net

In [11]:
for alt in alt_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+alt}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(alt)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")
for exc in exc_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+exc}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(exc)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")

  print()
  print()
promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
sub_gpt_response = ask(promptSubScenario, client, model)
print(prompt_all)
print("\all flow gen sub flow:"+sub_gpt_response +"\n")


basic flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system redirects learner to History page.
Step 3: Learner selects "Day History" on History page.
Step 4: The system redirects learner to a page show how many days learner has learned continuously in number. If learner break their chain, the number reset to one.

ll flow gen sub flow:Based on the provided information, there is no alternative or exception flow explicitly mentioned. However, I will infer a potential alternative or exception flow based on the main flow described and create scenarios that could lead a user from the main flow to this inferred alternative or exception flow.

Inferred Alternative/Exception Flow:
A scenario where the system fails to redirect the learner to the "Day History" page due to a system error or connectivity issue after they select "Day History" on the History page.

Scenarios:
1. "System Error Redirecting to Day History Page" - This scenario occurs when there is a system error or

In [12]:
for alt in alt_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+alt}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(alt)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")
for exc in exc_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+exc}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(exc)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")

  print()
  print()
promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
sub_gpt_response = ask(promptSubScenario, client, model)
print(prompt_all)
print("\all flow gen sub flow:"+sub_gpt_response +"\n")


basic flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system redirects learner to History page.
Step 3: Learner selects "Day History" on History page.
Step 4: The system redirects learner to a page show how many days learner has learned continuously in number. If learner break their chain, the number reset to one.

ll flow gen sub flow:Based on the provided information, there is no alternative or exception flow mentioned directly. However, considering the nature of software testing and the details given, I will infer a potential alternative or exception flow related to the main flow described. The main flow focuses on tracking the continuous learning days of a learner, which implies a potential for an alternative or exception flow if the learner breaks their learning chain.

Alternative or Exception Flow:
1. Learner breaks their continuous learning chain.

Predicted Scenarios Leading from Main Flow to Alternative or Exception Flow:
1. Learner fails to log in and lea